In [2]:
import os
import sys
mod_path = os.path.abspath(os.path.join('..'))
if mod_path not in sys.path:
    sys.path.append(mod_path)

In [84]:
from pprint import pprint as pp
import pandas as pd
import numpy as np
import pickle
from pathlib import Path
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectKBest

In [4]:
from tools.feature_format import featureFormat, targetFeatureSplit
from final_project.tester import dump_classifier_and_data

In [5]:
pwd_ = %pwd
pwd_ = Path(pwd_)
pwd_.joinpath('tools/feature_format.py')

WindowsPath('E:/Users/Trenton J. McKinney/PycharmProjects/ud120-projects/tools/feature_format.py')

# Task 1: Select Features
#### features_list is a list of strings, each of which is a feature name.
#### The first feature must be "poi".

In [6]:
financial_features = ['salary', 'deferral_payments', 'total_payments', 'loan_advances', 'bonus', 'restricted_stock_deferred',
                      'deferred_income', 'total_stock_value', 'expenses', 'exercised_stock_options', 'other',
                      'long_term_incentive', 'restricted_stock', 'director_fees']

email_features = ['to_messages', 'email_address', 'from_poi_to_this_person', 'from_messages', 'from_this_person_to_poi', 'shared_receipt_with_poi']

poi_label = ['poi']

In [7]:
feature_list = []
feature_list.extend(poi_label)
feature_list.extend(financial_features)
feature_list.extend(email_features)
feature_list

['poi',
 'salary',
 'deferral_payments',
 'total_payments',
 'loan_advances',
 'bonus',
 'restricted_stock_deferred',
 'deferred_income',
 'total_stock_value',
 'expenses',
 'exercised_stock_options',
 'other',
 'long_term_incentive',
 'restricted_stock',
 'director_fees',
 'to_messages',
 'email_address',
 'from_poi_to_this_person',
 'from_messages',
 'from_this_person_to_poi',
 'shared_receipt_with_poi']

In [86]:
len(feature_list)

21

## Load Dataset

In [48]:
# Load the dictionary containing the dataset
with open('final_project/final_project_dataset_unix.pkl', 'rb') as data_file:
    data_dict = pickle.load(data_file)

## Explore Dataset

In [49]:
df = pd.DataFrame.from_dict(data_dict, orient='index')
df = df.replace('NaN', np.nan)  # can't use the isnull method with 'NaN'
pd.set_option('max_rows', 150)
pd.set_option('max_columns', 21)
df

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,expenses,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
ALLEN PHILLIP K,201955.0,2902.0,2869717.0,4484442.0,NaN,4175000.0,phillip.allen@enron.com,-126027.0,-3081055.0,1729541.0,13868.0,47.0,1729541.0,2195.0,152.0,65.0,False,304805.0,1407.0,126027.0,NaN
BADUM JAMES P,NaN,NaN,178980.0,182466.0,NaN,NaN,NaN,NaN,NaN,257817.0,3486.0,NaN,257817.0,NaN,NaN,NaN,False,NaN,NaN,NaN,NaN
BANNANTINE JAMES M,477.0,566.0,NaN,916197.0,NaN,NaN,james.bannantine@enron.com,-560222.0,-5104.0,5243487.0,56301.0,39.0,4046157.0,29.0,864523.0,0.0,False,NaN,465.0,1757552.0,NaN
BAXTER JOHN C,267102.0,NaN,1295738.0,5634343.0,NaN,1200000.0,NaN,NaN,-1386055.0,10623258.0,11200.0,NaN,6680544.0,NaN,2660303.0,NaN,False,1586055.0,NaN,3942714.0,NaN
BAY FRANKLIN R,239671.0,NaN,260455.0,827696.0,NaN,400000.0,frank.bay@enron.com,-82782.0,-201641.0,63014.0,129142.0,NaN,NaN,NaN,69.0,NaN,False,NaN,NaN,145796.0,NaN
BAZELIDES PHILIP J,80818.0,NaN,684694.0,860136.0,NaN,NaN,NaN,NaN,NaN,1599641.0,NaN,NaN,1599641.0,NaN,874.0,NaN,False,93750.0,NaN,NaN,NaN
BECK SALLY W,231330.0,7315.0,NaN,969068.0,NaN,700000.0,sally.beck@enron.com,NaN,NaN,126027.0,37172.0,144.0,NaN,4343.0,566.0,386.0,False,NaN,2639.0,126027.0,NaN
BELDEN TIMOTHY N,213999.0,7991.0,2144013.0,5501630.0,NaN,5249999.0,tim.belden@enron.com,NaN,-2334434.0,1110705.0,17355.0,228.0,953136.0,484.0,210698.0,108.0,True,NaN,5521.0,157569.0,NaN
BELFER ROBERT,NaN,NaN,-102500.0,102500.0,NaN,NaN,NaN,44093.0,NaN,-44093.0,NaN,NaN,3285.0,NaN,NaN,NaN,False,NaN,NaN,NaN,3285.0
BERBERIAN DAVID,216582.0,NaN,NaN,228474.0,NaN,NaN,david.berberian@enron.com,NaN,NaN,2493616.0,11892.0,NaN,1624396.0,NaN,NaN,NaN,False,NaN,NaN,869220.0,NaN


In [59]:
describe_ = df.describe(percentiles=[])
describe_

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,restricted_stock_deferred,deferred_income,total_stock_value,expenses,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
count,9.500000e+01,86.000000,3.900000e+01,1.250000e+02,4.000000e+00,8.200000e+01,1.800000e+01,4.900000e+01,1.260000e+02,9.500000e+01,86.000000,1.020000e+02,86.000000,9.300000e+01,86.000000,6.600000e+01,86.000000,1.100000e+02,1.700000e+01
mean,5.621943e+05,2073.860465,1.642674e+06,5.081526e+06,4.196250e+07,2.374235e+06,1.664106e+05,-1.140475e+06,6.773957e+06,1.087289e+05,64.895349,5.987054e+06,608.790698,9.190650e+05,41.232558,1.470361e+06,1176.465116,2.321741e+06,1.668049e+05
std,2.716369e+06,2582.700981,5.161930e+06,2.906172e+07,4.708321e+07,1.071333e+07,4.201494e+06,4.025406e+06,3.895777e+07,5.335348e+05,86.979244,3.106201e+07,1841.033949,4.589253e+06,100.073111,5.942759e+06,1178.317641,1.251828e+07,3.198914e+05
min,4.770000e+02,57.000000,-1.025000e+05,1.480000e+02,4.000000e+05,7.000000e+04,-7.576788e+06,-2.799289e+07,-4.409300e+04,1.480000e+02,0.000000,3.285000e+03,12.000000,2.000000e+00,0.000000,6.922300e+04,2.000000,-2.604490e+06,3.285000e+03
50%,2.599960e+05,1211.000000,2.274490e+05,1.101393e+06,4.176250e+07,7.693750e+05,-1.469750e+05,-1.597920e+05,1.102872e+06,4.695000e+04,35.000000,1.310814e+06,41.000000,5.238200e+04,8.000000,4.420350e+05,740.500000,4.517400e+05,1.085790e+05
max,2.670423e+07,15149.000000,3.208340e+07,3.098866e+08,8.392500e+07,9.734362e+07,1.545629e+07,-8.330000e+02,4.345095e+08,5.235198e+06,528.000000,3.117640e+08,14368.000000,4.266759e+07,609.000000,4.852193e+07,5521.000000,1.303223e+08,1.398517e+06


In [60]:
df.isnull().sum()

salary                        51
to_messages                   60
deferral_payments            107
total_payments                21
loan_advances                142
bonus                         64
email_address                 35
restricted_stock_deferred    128
deferred_income               97
total_stock_value             20
expenses                      51
from_poi_to_this_person       60
exercised_stock_options       44
from_messages                 60
other                         53
from_this_person_to_poi       60
poi                            0
long_term_incentive           80
shared_receipt_with_poi       60
restricted_stock              36
director_fees                129
dtype: int64

In [61]:
df['poi'].value_counts()

False    128
True      18
Name: poi, dtype: int64

In [109]:
headers = list(df)
headers.sort()
headers

['bonus',
 'deferral_payments',
 'deferred_income',
 'director_fees',
 'email_address',
 'exercised_stock_options',
 'expenses',
 'from_messages',
 'from_poi_to_this_person',
 'from_this_person_to_poi',
 'loan_advances',
 'long_term_incentive',
 'other',
 'poi',
 'restricted_stock',
 'restricted_stock_deferred',
 'salary',
 'shared_receipt_with_poi',
 'to_messages',
 'total_payments',
 'total_stock_value']

In [89]:
len(headers)

21

In [63]:
list(df.index.values)

['ALLEN PHILLIP K',
 'BADUM JAMES P',
 'BANNANTINE JAMES M',
 'BAXTER JOHN C',
 'BAY FRANKLIN R',
 'BAZELIDES PHILIP J',
 'BECK SALLY W',
 'BELDEN TIMOTHY N',
 'BELFER ROBERT',
 'BERBERIAN DAVID',
 'BERGSIEKER RICHARD P',
 'BHATNAGAR SANJAY',
 'BIBI PHILIPPE A',
 'BLACHMAN JEREMY M',
 'BLAKE JR. NORMAN P',
 'BOWEN JR RAYMOND M',
 'BROWN MICHAEL',
 'BUCHANAN HAROLD G',
 'BUTTS ROBERT H',
 'BUY RICHARD B',
 'CALGER CHRISTOPHER F',
 'CARTER REBECCA C',
 'CAUSEY RICHARD A',
 'CHAN RONNIE',
 'CHRISTODOULOU DIOMEDES',
 'CLINE KENNETH W',
 'COLWELL WESLEY',
 'CORDES WILLIAM R',
 'COX DAVID',
 'CUMBERLAND MICHAEL S',
 'DEFFNER JOSEPH M',
 'DELAINEY DAVID W',
 'DERRICK JR. JAMES V',
 'DETMERING TIMOTHY J',
 'DIETRICH JANET R',
 'DIMICHELE RICHARD G',
 'DODSON KEITH',
 'DONAHUE JR JEFFREY M',
 'DUNCAN JOHN H',
 'DURAN WILLIAM D',
 'ECHOLS JOHN B',
 'ELLIOTT STEVEN',
 'FALLON JAMES B',
 'FASTOW ANDREW S',
 'FITZGERALD JAY L',
 'FOWLER PEGGY',
 'FOY JOE',
 'FREVERT MARK A',
 'FUGH JOHN L',
 'GAHN 

In [64]:
df.loc['LOCKHART EUGENE E']

salary                         NaN
to_messages                    NaN
deferral_payments              NaN
total_payments                 NaN
loan_advances                  NaN
bonus                          NaN
email_address                  NaN
restricted_stock_deferred      NaN
deferred_income                NaN
total_stock_value              NaN
expenses                       NaN
from_poi_to_this_person        NaN
exercised_stock_options        NaN
from_messages                  NaN
other                          NaN
from_this_person_to_poi        NaN
poi                          False
long_term_incentive            NaN
shared_receipt_with_poi        NaN
restricted_stock               NaN
director_fees                  NaN
Name: LOCKHART EUGENE E, dtype: object

In [65]:
df.loc['THE TRAVEL AGENCY IN THE PARK']

salary                          NaN
to_messages                     NaN
deferral_payments               NaN
total_payments               362096
loan_advances                   NaN
bonus                           NaN
email_address                   NaN
restricted_stock_deferred       NaN
deferred_income                 NaN
total_stock_value               NaN
expenses                        NaN
from_poi_to_this_person         NaN
exercised_stock_options         NaN
from_messages                   NaN
other                        362096
from_this_person_to_poi         NaN
poi                           False
long_term_incentive             NaN
shared_receipt_with_poi         NaN
restricted_stock                NaN
director_fees                   NaN
Name: THE TRAVEL AGENCY IN THE PARK, dtype: object

In [66]:
df.loc['TOTAL']

salary                       2.67042e+07
to_messages                          NaN
deferral_payments            3.20834e+07
total_payments               3.09887e+08
loan_advances                 8.3925e+07
bonus                        9.73436e+07
email_address                        NaN
restricted_stock_deferred   -7.57679e+06
deferred_income             -2.79929e+07
total_stock_value             4.3451e+08
expenses                      5.2352e+06
from_poi_to_this_person              NaN
exercised_stock_options      3.11764e+08
from_messages                        NaN
other                        4.26676e+07
from_this_person_to_poi              NaN
poi                                False
long_term_incentive          4.85219e+07
shared_receipt_with_poi              NaN
restricted_stock             1.30322e+08
director_fees                1.39852e+06
Name: TOTAL, dtype: object

# Task 2: Remove Outliers

In [67]:
outliers = ['THE TRAVEL AGENCY IN THE PARK', 'LOCKHART EUGENE E', 'TOTAL']
for value in outliers:
    del data_dict[value]

In [68]:
len(data_dict)

143

# Task 3: Create New Features

In [77]:
# In order to perform calculations
df = df.replace(np.nan, 0.0)
df['poi'] = df['poi'].replace([False, True], [0.0, 1.0])

In [79]:
df.head(8)

,salary,to_messages,deferral_payments,total_payments,loan_advances,bonus,email_address,restricted_stock_deferred,deferred_income,total_stock_value,expenses,from_poi_to_this_person,exercised_stock_options,from_messages,other,from_this_person_to_poi,poi,long_term_incentive,shared_receipt_with_poi,restricted_stock,director_fees
ALLEN PHILLIP K,201955.0,2902.0,2869717.0,4484442.0,0.0,4175000.0,phillip.allen@enron.com,-126027.0,-3081055.0,1729541.0,13868.0,47.0,1729541.0,2195.0,152.0,65.0,0.0,304805.0,1407.0,126027.0,0.0
BADUM JAMES P,0.0,0.0,178980.0,182466.0,0.0,0.0,0,0.0,0.0,257817.0,3486.0,0.0,257817.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
BANNANTINE JAMES M,477.0,566.0,0.0,916197.0,0.0,0.0,james.bannantine@enron.com,-560222.0,-5104.0,5243487.0,56301.0,39.0,4046157.0,29.0,864523.0,0.0,0.0,0.0,465.0,1757552.0,0.0
BAXTER JOHN C,267102.0,0.0,1295738.0,5634343.0,0.0,1200000.0,0,0.0,-1386055.0,10623258.0,11200.0,0.0,6680544.0,0.0,2660303.0,0.0,0.0,1586055.0,0.0,3942714.0,0.0
BAY FRANKLIN R,239671.0,0.0,260455.0,827696.0,0.0,400000.0,frank.bay@enron.com,-82782.0,-201641.0,63014.0,129142.0,0.0,0.0,0.0,69.0,0.0,0.0,0.0,0.0,145796.0,0.0
BAZELIDES PHILIP J,80818.0,0.0,684694.0,860136.0,0.0,0.0,0,0.0,0.0,1599641.0,0.0,0.0,1599641.0,0.0,874.0,0.0,0.0,93750.0,0.0,0.0,0.0
BECK SALLY W,231330.0,7315.0,0.0,969068.0,0.0,700000.0,sally.beck@enron.com,0.0,0.0,126027.0,37172.0,144.0,0.0,4343.0,566.0,386.0,0.0,0.0,2639.0,126027.0,0.0
BELDEN TIMOTHY N,213999.0,7991.0,2144013.0,5501630.0,0.0,5249999.0,tim.belden@enron.com,0.0,-2334434.0,1110705.0,17355.0,228.0,953136.0,484.0,210698.0,108.0,1.0,0.0,5521.0,157569.0,0.0


In [80]:
df.values

array([[201955.0, 2902.0, 2869717.0, ..., 1407.0, 126027.0, 0.0],
       [0.0, 0.0, 178980.0, ..., 0.0, 0.0, 0.0],
       [477.0, 566.0, 0.0, ..., 465.0, 1757552.0, 0.0],
       ...,
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0],
       [158403.0, 0.0, 0.0, ..., 0.0, 3576206.0, 0.0],
       [0.0, 0.0, 0.0, ..., 0.0, 0.0, 0.0]], dtype=object)

In [110]:
headers.remove('poi')
headers.remove('email_address')

In [121]:
k_best = SelectKBest(k='all')
k_best.fit(df[financial_features].values, df['poi'].values)
score = k_best.scores_
score_chart = list(zip(financial_features, score))
score_chart_df = pd.DataFrame(score_chart, columns=['Feature', 'Score'])

In [122]:
score_chart_df

,Feature,Score
0,salary,0.000057
1,deferral_payments,0.235515
2,total_payments,0.357679
3,loan_advances,2.545253
4,bonus,0.081305
5,restricted_stock_deferred,0.004141
6,deferred_income,0.225532
7,total_stock_value,0.171227
8,expenses,0.012878
9,exercised_stock_options,0.234424


In [82]:
resources = pd.read_csv('resources.csv')

In [83]:
resources

,title,author,url
0,An Introduction to Feature Selection,Jason Brownlee,https://machinelearningmastery.com/an-introduc...
1,A Feature Selection Tool for Machine Learning ...,William Koehrsen,https://towardsdatascience.com/a-feature-selec...
2,Machine Learning with Scikit-Learn - 42 - Auto...,Cristi Vlad,https://www.youtube.com/watch?v=RXFnwCRb-is
3,scikit-learn v0.19.2 Section 1.13 Feature Sele...,NaN,http://scikit-learn.org/stable/modules/feature...
